In [59]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Union, Callable
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import CapturedException

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

BATCH_DATE_TO_USE = '2023-08-24' # used for filenames throught the notebook

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of neuropy.core.session.Formats.BaseDataSessionFormats failed: Traceback (most recent call last):
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__ to DataSe

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder



{'DataSessionFormatBaseRegisteredClass': neuropy.core.session.Formats.BaseDataSessionFormats.DataSessionFormatBaseRegisteredClass}

In [2]:
# active_global_batch_result_filename='global_batch_result_2023-07-12.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07_extra.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-20.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-21_Greatlakes.pkl'
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'
debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')]
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
# with set_posix_windows():
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     # display(batch_progress_df)
#     # display(good_only_batch_progress_df)
#     display(batch_progress_df)

Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_2023-08-24.pkl... done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,279,40,224,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,46,179,40,142,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,37,34,55,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,False,True
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/KDIBA/pin01/one/redundant,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/KDIBA/pin01/one/showclus,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/KDIBA/pin01/one/sleep,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False


In [34]:
batch_progress_df[IdentifyingContext._get_session_context_keys()].to_clipboard(index=False)

In [62]:
# def try_extract_date_from_session_name(session_name) -> Optional[datetime]:
#     """ 
#     From the 'session_name' column in the provided data, we can observe two different formats used to specify the date:

#     Format 1: Dates with the pattern YYYY-M-D_H-M-S (e.g., "2006-6-07_11-26-53").
#     Format 2: Dates with the pattern MM-DD_H-M-S (e.g., "11-02_17-46-44").
    
#     """
#     # Remove any non-digit prefixes or suffixes before parsing. Handles 'fet11-01_12-58-54'

#     # Check for any non-digit prefix
#     if re.match(r'^\D+', session_name):
#         print(f"WARN: Removed prefix from session_name: {session_name}")
#         session_name = re.sub(r'^\D*', '', session_name)

#     # Check for any non-digit suffix
#     if re.search(r'\D+$', session_name):
#         print(f"WARN: Removed suffix from session_name: {session_name}")
#         session_name = re.sub(r'\D*$', '', session_name)


#     # Try Format 1 (YYYY-M-D_H-M-S)
#     date_match1 = re.search(r'\d{4}-\d{1,2}-\d{1,2}_\d{1,2}-\d{1,2}-\d{1,2}', session_name)
#     if date_match1:
#         date_str1 = date_match1.group().replace('_', ' ')
#         return pd.to_datetime(date_str1, format='%Y-%m-%d %H-%M-%S', errors='coerce')

#     # Try Format 2 (MM-DD_H-M-S)
#     date_match2 = re.search(r'\d{1,2}-\d{1,2}_\d{1,2}-\d{1,2}-\d{1,2}', session_name)
#     if date_match2:
#         date_str2 = "2000-" + session_name.split('_')[0] # Assuming year 2000
#         time_str2 = session_name.split('_')[1].replace('-', ':')
#         full_str2 = date_str2 + ' ' + time_str2
#         return pd.to_datetime(full_str2, format='%Y-%m-%d %H:%M:%S', errors='coerce')

#     print(f"WARN: Could not parse date from session_name: {session_name} for any known format.")
#     return None

# from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
# ## Tries to get the appropriate class using its self.format_name and compute its context
# active_data_mode_registered_class = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[self.format_name]
# self.session_context = active_data_mode_registered_class.parse_session_basepath_to_context(self.basepath)

df = deepcopy(batch_progress_df)
# Apply the extract_date function to the 'session_name' column to create a new 'session_date' column
# parsed_session_dates = df['session_name'].apply(extract_date_from_session_name).to_numpy()
df['session_date'] = df['session_name'].apply(IdentifyingContext.try_extract_date_from_session_name)

# df['session_date'] = df[['format_name','session_name']].itertuples(index=False).apply(lambda row: DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[row.format_name].extract_date_from_session_name(row.session_name))

# df['session_date'] = df[['format_name','session_name']].itertuples(index=False) # .apply(lambda row: DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[row.format_name].extract_date_from_session_name(row.session_name))




# [IdentifyingContext.try_extract_date_from_session_name(row.session_name) for row in df[['format_name','session_name']].itertuples(index=False)]

# .apply(lambda a_format_name, a_session_name: DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[a_format_name].extract_date_from_session_name(a_session_name))


# # df['session_date']
# df
# # parsed_session_dates

with pd.option_context('display.max_rows', 100):
    display(df)

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations,session_date
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,False,True,2006-06-07 11:26:53
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,279,40,224,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,46,179,40,142,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True,2006-06-09 01:22:43
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23...,False,True,2006-06-09 03:23:37
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,37,34,55,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True,2006-06-12 15:55:31
5,kdiba,gor01,one,2006-6-13_14-42-6,kdiba_gor01_one_2006-6-13_14-42-6,/media/MAX/Data/KDIBA/gor01/one/2006-6-13_14-42-6,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,/media/MAX/Data/KDIBA/gor01/one/2006-6-13_14-4...,/media/MAX/Data/KDIBA/gor01/one/2006-6-13_14-4...,False,True,2006-06-13 14:42:06
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,42,212,40,333,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,True,True,2006-06-07 16:40:19
7,kdiba,gor01,two,2006-6-08_15-46-47,kdiba_gor01_two_2006-6-08_15-46-47,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_15-4...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_15-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_15-4...,False,True,2006-06-08 15:46:47
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,SessionBatchProgress.COMPLETED,None,40,45,40,62,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,True,True,2006-06-08 21:16:25
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,SessionBatchProgress.COMPLETED,None,51,106,43,406,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,True,True,2006-06-09 22:24:40


In [54]:
DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()

{}

In [ ]:

batch_progress_df.session_name.to_clipboard(index=False)
session_name
2006-6-07_11-26-53
2006-6-08_14-26-15
2006-6-09_1-22-43
2006-6-09_3-23-37
2006-6-12_15-55-31
2006-6-13_14-42-6
2006-6-07_16-40-19
2006-6-08_15-46-47
2006-6-08_21-16-25
2006-6-09_22-24-40
2006-6-12_16-53-46
2006-6-13_15-22-3
2006-4-09_17-29-30
2006-4-10_12-25-50
2006-4-10_21-2-40
2006-4-11_15-16-59
2006-4-12_14-39-31
2006-4-12_17-53-55
2006-4-16_15-12-23
2006-4-17_12-33-47
2006-4-18_13-6-1
2006-4-18_15-23-32
2006-4-19_13-34-40
2006-4-19_16-48-9
2006-4-21_10-24-35
2006-4-25_14-28-51
2006-4-25_17-17-6
2006-4-26_13-22-13
2006-4-27_14-43-12
2006-4-28_12-17-27
2006-4-28_16-48-29
2006-4-09_16-40-54
2006-4-10_12-58-3
2006-4-10_19-11-57
2006-4-11_12-48-38
2006-4-11_16-2-46
2006-4-12_14-59-23
2006-4-12_15-25-59
2006-4-16_14-49-24
2006-4-16_18-47-52
2006-4-17_12-52-15
2006-4-18_13-28-57
2006-4-18_15-38-2
2006-4-19_13-50-7
2006-4-19_16-37-40
2006-4-21_11-19-2
2006-4-25_13-20-55
2006-4-25_17-33-28
2006-4-26_13-51-50
2006-4-27_18-21-57
2006-4-28_12-38-13
2006-4-28_17-6-14
11-02_17-46-44
11-02_19-28-0
11-03_12-3-25
11-03_21-26-8
11-05_19-26-43
11-09_11-43-50
11-09_12-15-3
11-09_21-17-16
11-09_22-4-5
11-19_12-35-59
11-19_13-2-0
11-19_13-55-7
fet11-01_12-58-54
fet11-03_11-0-53
fet11-03_20-28-3
fet11-04_21-20-3



# Run Batch Executions/Computations

In [3]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # Bad?
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')]


included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
included_session_batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,279,40,224,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,46,179,40,142,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,37,34,55,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,42,212,40,333,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,True,True
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,SessionBatchProgress.COMPLETED,None,40,45,40,62,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,True,True
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,SessionBatchProgress.COMPLETED,None,51,106,43,406,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,True,True
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,SessionBatchProgress.COMPLETED,None,41,59,40,48,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,True,True
12,kdiba,vvp01,one,2006-4-09_17-29-30,kdiba_vvp01_one_2006-4-09_17-29-30,/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-2...,SessionBatchProgress.COMPLETED,None,51,45,42,62,True,/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-2...,/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-2...,/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-2...,True,True
13,kdiba,vvp01,one,2006-4-10_12-25-50,kdiba_vvp01_one_2006-4-10_12-25-50,/media/MAX/Data/KDIBA/vvp01/one/2006-4-10_12-2...,SessionBatchProgress.COMPLETED,None,50,22,42,17,True,/media/MAX/Data/KDIBA/vvp01/one/2006-4-10_12-2...,/media/MAX/Data/KDIBA/vvp01/one/2006-4-10_12-2...,/media/MAX/Data/KDIBA/vvp01/one/2006-4-10_12-2...,True,True
31,kdiba,vvp01,two,2006-4-09_16-40-54,kdiba_vvp01_two_2006-4-09_16-40-54,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,SessionBatchProgress.COMPLETED,None,48,27,50,20,True,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,True,True


In [5]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 
included_h5_paths

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata

[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]
[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]

[datetime.datetime(2023, 8, 24, 10, 0, 51, 303126),
 datetime.datetime(2023, 8, 24, 9, 56, 27, 525824),
 datetime.datetime(2023, 8, 8, 14, 45, 30, 262058),
 datetime.datetime(2023, 8, 24, 10, 18, 6, 394212),
 datetime.datetime(2023, 8, 24, 10, 10, 28, 491735),
 datetime.datetime(2023, 8, 24, 10, 27, 36, 708793),
 datetime.datetime(2023, 8, 24, 10, 22, 49, 318501),
 datetime.datetime(2023, 8, 24, 10, 30, 20, 101959),
 datetime.datetime(2023, 8, 24, 10, 36, 15, 896319),
 datetime.datetime(2023, 8, 24, 10, 38, 2, 41430),
 datetime.datetime(2023, 8, 24, 10, 43, 4, 588744),
 datetime.datetime(2023, 8, 24, 10, 45, 43, 449910),
 datetime.datetime(2023, 8, 24, 10, 46, 49, 658978),
 datetime.datetime(2023, 8, 24, 10, 48, 23, 494076),
 datetime.datetime(2023, 8, 24, 10, 54, 22, 105450)]

In [6]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import H5FileReference, H5ExternalLinkBuilder, ExternallyReferencedItem
# restore native column types:
from neuropy.core.neurons import NeuronType
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

session_short_names: List[str] = [a_ctxt.get_description(separator='_') for a_ctxt in included_session_contexts] # 'kdiba.gor01.one.2006-6-08_14-26-15'
session_group_keys: List[str] = [("/" + a_ctxt.get_description(separator="/", include_property_names=False)) for a_ctxt in included_session_contexts] # 'kdiba/gor01/one/2006-6-08_14-26-15'
# Particular Table Keys:
neuron_replay_stats_df_table_keys = [f"{session_group_key}/global_computations/jonathan_fr_analysis/neuron_replay_stats_df/table" for session_group_key in session_group_keys]
neuron_identities_table_keys = [f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys]
long_short_fr_indicies_analysis_table_keys = [f"{session_group_key}/global_computations/long_short_fr_indicies_analysis/table" for session_group_key in session_group_keys]

a_loader = H5ExternalLinkBuilder.init_from_file_lists(file_list=included_h5_paths, table_key_list=[f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys], short_name_list=session_short_names)

In [ ]:

# neuron_identities_table_keys = [ExternallyReferencedItem(f"{session_group_key}/neuron_identities/table", '/neuron_identities') for session_group_key in session_group_keys]
destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file.h5', referential_group_key='neuron_identities', table_key_list=neuron_identities_table_keys, fail_on_exception=False)
external_file_links


In [ ]:

destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file_new.h5', referential_group_key='long_short_fr_indicies_analysis', table_key_list=long_short_fr_indicies_analysis_table_keys, fail_on_exception=False)
external_file_links


In [ ]:
# destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file_new.h5', referential_group_key='neuron_replay_stats_df', table_key_list=neuron_replay_stats_df_table_keys, fail_on_exception=False)
# external_file_links

In [7]:
_out_table = a_loader.load_and_consolidate(table_key_list=neuron_replay_stats_df_table_keys, fail_on_exception=False)
_out_table = HDF_Converter.general_post_load_restore_table_as_needed(_out_table)
# Drop columns: 'neuron_IDX', 'has_short_pf' and 3 other columns
_out_table = _out_table.drop(columns=['neuron_IDX', 'has_short_pf', 'has_na', 'has_long_pf', 'index'])
_out_table

failed for file path: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/jonathan_fr_analysis/neuron_replay_stats_df/table. wth exception HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /usr/local/hdf5/lib/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1990, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 614, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,...,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,neuron_type,aclu,session_uid,neuron_uid
0,kdiba,gor01,one,2006-6-08_14-26-15,212.160000,NaN,LEFT_ONLY,0.419960,0.114724,-0.305237,...,4.773420,NaN,NaN,31,31,0,pyr,2,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2
1,kdiba,gor01,one,2006-6-08_14-26-15,136.160000,198.160000,SHARED,0.275226,1.227759,0.952532,...,4.986733,5.454890,0.468157,71,19,52,pyr,3,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,178.160000,RIGHT_ONLY,0.117397,0.834271,0.716874,...,NaN,7.565518,NaN,67,0,67,pyr,4,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4
3,kdiba,gor01,one,2006-6-08_14-26-15,98.160000,202.160000,SHARED,2.090985,1.844885,-0.246099,...,8.092135,7.583226,-0.508909,163,91,72,pyr,5,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,26.129319,12.892497,-13.236821,...,68.830796,27.813785,-41.017012,495,279,216,intr,6,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,kdiba,pin01,one,fet11-01_12-58-54,179.403791,177.403791,SHARED,0.892852,1.027642,0.134789,...,6.178167,7.307386,1.129219,91,52,39,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28
831,kdiba,pin01,one,fet11-01_12-58-54,59.403791,107.403791,SHARED,0.837484,0.766268,-0.071216,...,5.987154,6.376664,0.389510,164,121,43,pyr,29,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29
832,kdiba,pin01,one,fet11-01_12-58-54,119.403791,127.403791,SHARED,0.712330,0.577649,-0.134681,...,6.552028,6.036531,-0.515497,88,56,32,pyr,30,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30
833,kdiba,pin01,one,fet11-01_12-58-54,217.403791,33.403791,SHARED,0.902182,1.016473,0.114291,...,5.656711,5.564682,-0.092028,115,80,35,pyr,31,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31


In [18]:
# How many of each track_membership type in each session?
_gdf = _out_table.groupby(['session_uid', 'track_membership']).agg(index_count=('neuron_uid', 'count')).reset_index()
_gdf.to_clipboard(excel=True, index=False)

In [24]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_out_value_counts = _out_table.value_counts(subset=['format_name', 'animal', 'session_name','track_membership'], normalize=False, sort=False, ascending=True, dropna=True)
# pd.DataFrame(_out_value_counts)
# _out_value_counts.to_dict()
_out_value_counts


format_name  animal  session_name        track_membership
kdiba        gor01   2006-6-07_16-40-19  LEFT_ONLY            3
                                         RIGHT_ONLY           4
                                         SHARED              57
                     2006-6-08_14-26-15  LEFT_ONLY           12
                                         RIGHT_ONLY           8
                                         SHARED              88
                     2006-6-08_21-16-25  LEFT_ONLY           11
                                         RIGHT_ONLY           4
                                         SHARED              87
                     2006-6-09_1-22-43   LEFT_ONLY            8
                                         RIGHT_ONLY           5
                                         SHARED              90
                     2006-6-09_22-24-40  LEFT_ONLY            6
                                         RIGHT_ONLY          10
                                         SHARE

In [27]:
_out_value_counts_df = _out_value_counts.reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'track_membership', 'count']
_out_value_counts_df

,format_name,animal,session_name,track_membership,count
0,kdiba,gor01,2006-6-07_16-40-19,LEFT_ONLY,3
1,kdiba,gor01,2006-6-07_16-40-19,RIGHT_ONLY,4
2,kdiba,gor01,2006-6-07_16-40-19,SHARED,57
3,kdiba,gor01,2006-6-08_14-26-15,LEFT_ONLY,12
4,kdiba,gor01,2006-6-08_14-26-15,RIGHT_ONLY,8
5,kdiba,gor01,2006-6-08_14-26-15,SHARED,88
6,kdiba,gor01,2006-6-08_21-16-25,LEFT_ONLY,11
7,kdiba,gor01,2006-6-08_21-16-25,RIGHT_ONLY,4
8,kdiba,gor01,2006-6-08_21-16-25,SHARED,87
9,kdiba,gor01,2006-6-09_1-22-43,LEFT_ONLY,8


In [26]:
I get the number of LEFT_ONLY, RIGHT_ONLY, and SHARED entries for each session_name for each animal for each format_name using: `_out_value_counts = _out_table.value_counts(subset=['format_name', 'animal', 'session_name','track_membership'], normalize=False, sort=False, ascending=True, dropna=True)`. This returns the pd.Series:
```
format_name  animal  session_name        track_membership
kdiba        gor01   2006-6-07_16-40-19  LEFT_ONLY            3
                                         RIGHT_ONLY           4
                                         SHARED              57
                     2006-6-08_14-26-15  LEFT_ONLY           12
                                         RIGHT_ONLY           8
                                         SHARED              88
                     2006-6-08_21-16-25  LEFT_ONLY           11
                                         RIGHT_ONLY           4
                                         SHARED              87
                     2006-6-09_1-22-43   LEFT_ONLY            8
                                         RIGHT_ONLY           5
                                         SHARED              90
                     2006-6-09_22-24-40  LEFT_ONLY            6
                                         RIGHT_ONLY          10
                                         SHARED              80
                     2006-6-12_16-53-46  LEFT_ONLY            5
                                         RIGHT_ONLY           2
                                         SHARED              52
             pin01   11-02_17-46-44      LEFT_ONLY            5
                                         RIGHT_ONLY           4
                                         SHARED              33
                     11-02_19-28-0       LEFT_ONLY            4
                                         SHARED              34
                     11-03_12-3-25       RIGHT_ONLY           3
                                         SHARED              26
                     fet11-01_12-58-54   LEFT_ONLY            4
                                         RIGHT_ONLY           1
                                         SHARED              26
             vvp01   2006-4-09_16-40-54  LEFT_ONLY            4
                                         RIGHT_ONLY           1
                                         SHARED              30
                     2006-4-09_17-29-30  LEFT_ONLY            6
                                         RIGHT_ONLY           3
                                         SHARED              35
                     2006-4-10_12-25-50  LEFT_ONLY            2
                                         RIGHT_ONLY           2
                                         SHARED              34
                     2006-4-10_12-58-3   LEFT_ONLY            7
                                         RIGHT_ONLY           2
                                         SHARED              37
dtype: int64
```
How can I convert this to a dataframe with exhaustive entries, meaning an entry for each line in the returned result?


SyntaxError: invalid syntax (3556040754.py, line 1)

In [ ]:
# Convert the muilti-index pd.Series into an exhaustive dataframe:

active_aclu_binned_time_multiindex = unit_specific_bin_specific_spike_counts.index
active_unique_aclu_values = np.unique(active_aclu_binned_time_multiindex.get_level_values('aclu'))
unit_specific_binned_spike_counts = np.array([unit_specific_bin_specific_spike_counts[aclu].values for aclu in active_unique_aclu_values]).T # (85841, 40)
if debug_print:
	print(f'np.shape(unit_specific_spike_counts): {np.shape(unit_specific_binned_spike_counts)}') # np.shape(unit_specific_spike_counts): (40, 85841)
unit_specific_binned_spike_counts = pd.DataFrame(unit_specific_binned_spike_counts, columns=active_unique_aclu_values, index=time_bin_indicies)

for (xbin_label, ybin_label), count in _out_value_counts.iteritems():
	if debug_print:
		print(f'xbin_label: {xbin_label}, ybin_label: {ybin_label}, count: {count}')
	
	# last_occupancy_matrix[xbin_label, ybin_label] += count
	try:
		last_occupancy_matrix[xbin_label-1, ybin_label-1] += count
		# last_occupancy_matrix[xbin_label, ybin_label] += count
	except IndexError as e:
		print(f'e: {e}\n active_current_pos_df: {np.shape(active_current_pos_df)}, current_bin_counts: {np.shape(_out_value_counts)}\n last_occupancy_matrix: {np.shape(last_occupancy_matrix)}\n count: {count}')
		raise e
	

In [ ]:
_out_table_long_short_fr_indicies_analysis = a_loader.load_and_consolidate(table_key_list=long_short_fr_indicies_analysis_table_keys, fail_on_exception=False)
_out_table_long_short_fr_indicies_analysis = HDF_Converter.general_post_load_restore_table_as_needed(_out_table_long_short_fr_indicies_analysis)
_out_table_long_short_fr_indicies_analysis

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=_out_table_long_short_fr_indicies_analysis, num_sessions=0)


In [ ]:
# TODO: More robust and automatic attempt to restore native column types:
# _out_table = HDF_Converter.HDFConvertableEnum.restore_hdf_dataframe_column_original_type(_out_table, column_name='track_membership') # AttributeError: 'int' object has no attribute 'lower'
# SplitPartitionMembership.restore_hdf_dataframe_column_original_type(_out_table, column_name='track_membership')
# _out_table = HDF_Converter.HDFConvertableEnum.restore_hdf_dataframe_column_original_type(_out_table, column_name='neuron_type')
# _out_table["neuron_type"] = _out_table["neuron_type"].apply(lambda x: NeuronType.from_hdf_coding_string(x)).astype(object) #.astype(str) # interestingly this leaves the dtype of this column as 'category' still, but _spikes_df["cell_type"].to_numpy() returns the correct array of objects... this is better than it started before saving, but not the same. 
_out_table

In [ ]:
_out_table.groupby(['session_uid', 'track_membership', 'neuron_type']).agg(neuron_uid_count=('neuron_uid', 'count')).reset_index()

In [ ]:
import sys
from pyphocorehelpers.print_helpers import CapturedException
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
import tables as tb
from copy import deepcopy
from neuropy.core import Epoch
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler


In [ ]:

@define(slots=False, repr=False)
class HDFSpecificBatchSessionCompletionHandler(BatchSessionCompletionHandler):
    """ handles completion of a single session's batch processing. 

    Allows accumulating results across sessions and runs.

    
    Usage:
        from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
        
    """

    # Completion Result object returned from callback ____________________________________________________________________ #
 
    # Cross-session Results:
    across_sessions_instantaneous_fr_dict: dict = Factory(dict) # Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation

    ## Main function that's called with the complete pipeline:
    def on_complete_success_execution_session(self, global_data_root_parent_path, curr_session_context, curr_session_basedir, curr_active_pipeline) -> PipelineCompletionResult:
        """ called when the execute_session completes like:
            `post_run_callback_fn_output = post_run_callback_fn(curr_session_context, curr_session_basedir, curr_active_pipeline)`
            
            Meant to be assigned like:
            , post_run_callback_fn=_on_complete_success_execution_session
            
            Captures nothing.
            
            from Spike3D.scripts.run_BatchAnalysis import _on_complete_success_execution_session
            
            
            LOGIC: really we want to recompute global whenever local is recomputed.
            
            
        """
        print(f'HDFProcessing.on_complete_success_execution_session(curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}, ...)')
        # print(f'curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}')
        long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
        # long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        # long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

        # Get existing laps from session:
        long_laps, short_laps, global_laps = [curr_active_pipeline.filtered_sessions[an_epoch_name].laps.as_epoch_obj() for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        long_replays, short_replays, global_replays = [Epoch(curr_active_pipeline.filtered_sessions[an_epoch_name].replay.epochs.get_valid_df()) for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        # short_laps.n_epochs: 40, n_long_laps.n_epochs: 40
        # short_replays.n_epochs: 6, long_replays.n_epochs: 8
        if self.debug_print:
            print(f'short_laps.n_epochs: {short_laps.n_epochs}, n_long_laps.n_epochs: {long_laps.n_epochs}')
            print(f'short_replays.n_epochs: {short_replays.n_epochs}, long_replays.n_epochs: {long_replays.n_epochs}')

        # ## Post Compute Validate 2023-05-16:
        try:
            was_updated = self.post_compute_validate(curr_active_pipeline)
        except Exception as e:
            exception_info = sys.exc_info()
            an_err = CapturedException(e, exception_info)
            print(f'self.post_compute_validate(...) failed with exception: {an_err}')
            raise 

        delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
        print(f'\t time since last computation: {delta_since_last_compute}')
        
        ## Save the pipeline since that's disabled by default now:
        try:
            curr_active_pipeline.save_pipeline(saving_mode=self.saving_mode, active_pickle_filename=self.override_session_computation_results_pickle_filename) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
        except Exception as e:
            ## TODO: catch/log saving error and indicate that it isn't saved.
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'ERROR SAVING PIPELINE for curr_session_context: {curr_session_context}. error: {e}')


        ## GLOBAL FUNCTION:
        if self.force_reload_all and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but self.force_reload_all was true. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True
            
        if was_updated and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but pipeline was_updated. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True


        ## GLOBAL FUNCTION:
        if self.force_reload_all and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but self.force_reload_all was true. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True
            
        if was_updated and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but pipeline was_updated. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True

        self.try_compute_global_computations_if_needed(curr_active_pipeline, curr_session_context=curr_session_context)

        ### Aggregate Outputs specific computations:

        ## Get some more interesting session properties:
        
        delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
        
        print(f'\t time since last computation: {delta_since_last_compute}')

        # Export the pipeline's HDF5:
        hdf5_output_path, hdf5_output_err = self.try_export_pipeline_hdf5_if_needed(curr_active_pipeline=curr_active_pipeline, curr_session_context=curr_session_context)
        
        # ## Specify the output file:
        # common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
        # print(f'common_file_path: {common_file_path}')
        # InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

        across_session_results_extended_dict = {}
            
        return PipelineCompletionResult(long_epoch_name=long_epoch_name, long_laps=long_laps, long_replays=long_replays,
                                           short_epoch_name=short_epoch_name, short_laps=short_laps, short_replays=short_replays,
                                           delta_since_last_compute=delta_since_last_compute,
                                           outputs_local={'pkl': curr_active_pipeline.pickle_path},
                                            outputs_global={'pkl': curr_active_pipeline.global_computation_results_pickle_path, 'hdf5': hdf5_output_path},
                                            across_session_results=across_session_results_extended_dict)
                                          


In [ ]:
# multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=2)
multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=2)

enable_full_pipeline_in_ram = False
# enable_full_pipeline_in_ram = True

# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
# included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()

# Only require sessions to have replay annotations:
# included_session_contexts = batch_progress_df[batch_progress_df['has_user_replay_annotations']]['context'].to_numpy().tolist()

# included_session_contexts = batch_progress_df['context'].to_numpy().tolist()[:4] # Only get the first 6
## Limit the contexts to run to the last N:
# included_session_contexts = included_session_contexts[3:6]

# ALL
included_session_contexts = included_session_contexts

# ## No filtering the sessions:
# included_session_contexts = None

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts


# No Reloading
result_handler = HDFSpecificBatchSessionCompletionHandler(force_reload_all=False,
                                                session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
                                                global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
                                                enable_full_pipeline_in_ram=enable_full_pipeline_in_ram,
                                                **multiprocessing_kwargs)


# # Forced Reloading:
# result_handler = HDFSpecificBatchSessionCompletionHandler(force_reload_all=True,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
#                                                 **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session


## Execute with the custom arguments.
active_computation_functions_name_includelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=True, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

In [ ]:
global_batch_run

In [ ]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 
included_h5_paths

In [ ]:
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata

def check_output_h5_files(included_file_paths, minimum_good_file_size_GB:float=0.01, include_too_small_files:bool=False):
	metadata = []

	for a_file in included_file_paths:
		# if not a_file.exists():
		fetched_metadata = get_file_metadata(a_file)
		if fetched_metadata is None:
			print(f'file {a_file} does not exist. Skipping.')
		else:
			if fetched_metadata['file_size'] < minimum_good_file_size_GB:
				print(f'WARN: file_size < {minimum_good_file_size_GB} for {a_file}!')
				if include_too_small_files:
					print(f'\t Continuing hesitantly.')
					metadata.append(fetched_metadata)
			else:
				# file size is reasonable:
				metadata.append(fetched_metadata)
			

	# pd.options.display.float_format = '{:.2f}'.format
	df = pd.DataFrame(metadata)
	# df.style.format("{:.1f}") # suppresses scientific notation display only for this dataframe. Alternatively: pd.options.display.float_format = '{:.2f}'.format
	# df['file_size'] = df['file_size'].round(decimals=2)

	# with pd.option_context('display.float_format', lambda x: f'{x:,.3f}'):
		# print(df)
	return df

df = check_output_h5_files(included_file_paths=included_h5_paths)
df

In [ ]:
df.dtypes
df.file_size.round(decimals=2)

In [ ]:

from pyphocorehelpers.Filesystem.metadata_helpers import 
[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]
[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]


def get_file_metadata(a_file):
    FilesystemMetadata.get_last_modified_time(a_file)

pd.DataFrame({'session_context':included_session_batch_progress_df['context'],
              'h5_paths': included_h5_paths,
              'modified_time':[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths],
              'global_result_modified_time':[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_global_computation_h5_paths],
             })

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import LoadedObjectPersistanceState

In [ ]:
batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df

In [ ]:
## 2023-08-10
# Add output modes for .pkl and .h5 files similar to figures. For GreatLakes Batch runs, allow output to the run folder.

# like the figures, file naming conventions should change for the different types.


In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=False)

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
sessions_with_results

In [ ]:
session_identifiers = included_session_contexts


In [ ]:
global_batch_run

In [ ]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

In [ ]:
# Combine across .h5 files:
import tables as tb
import h5py
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import H5ExternalLinkBuilder

# f'/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table'

session_identifiers, pkl_output_paths, hdf5_output_paths

file_path = f'output/test_external_links.h5'
a_global_computations_group_key: str = f"{session_group_key}/global_computations"
with tb.open_file(file_path, mode='w') as f: # this mode='w' is correct because it should overwrite the previous file and not append to it.
    a_global_computations_group = f.create_group(session_group_key, 'global_computations', title='the result of computations that operate over many or all of the filters in the session.', createparents=True)
    an_external_link = f.create_external_link(f'', n

In [ ]:


from typing import List
from attrs import define, field, Factory
import pandas as pd
import tables as tb

    
session_group_keys: List[str] = [("/" + a_ctxt.get_description(separator="/", include_property_names=False)) for a_ctxt in session_identifiers] # 'kdiba/gor01/one/2006-6-08_14-26-15'
# session_uid: str = session_context.get_description(separator="|", include_property_names=False)
neuron_identities_table_keys = [f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys]


a_loader = H5ExternalLinkBuilder(file_list=hdf5_output_paths, table_key_list=neuron_identities_table_keys)
a_loader

_out_table = a_loader.load_and_consolidate()
_out_table

# Extract output files from all completed sessions:

In [ ]:
# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}

session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

In [ ]:
session_identifiers

In [ ]:
pkl_output_paths

In [ ]:
hdf5_output_paths

In [ ]:
# Save output filelist:


def save_filelist_to_text_file(hdf5_output_paths, filelist_path: Path):
    _out_string = '\n'.join([str(a_file) for a_file in hdf5_output_paths])
    print(f'{_out_string}')
    print(f'saving out to "{filelist_path}"...')
    with open(filelist_path, 'w') as f:
        f.write(_out_string)
    return _out_string, filelist_path


h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)


In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from neuropy.core.epoch import Epoch

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
file_path

In [ ]:
global_batch_run.to_hdf(file_path,'/')

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# good_session_batch_outputs = global_batch_run.session_batch_outputs

sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
[a_ctxt.get_initialization_code_string() for a_ctxt in sessions_with_results]

# 2023-07-14 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
across_sessions_instantaneous_frs_list

In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

## Specify the output file:
common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
print(f'common_file_path: {common_file_path}')
InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
# Build the unique scatter plot dictionaries:
across_session_contexts = list(across_sessions_instantaneous_fr_dict.keys())
unique_animals = IdentifyingContext.find_unique_values(across_session_contexts)['animal'] # {'gor01', 'pin01', 'vvp01'}
# Get number of animals to plot
marker_list = [(5, i) for i in np.arange(len(unique_animals))] # [(5, 0), (5, 1), (5, 2)]
scatter_props = [{'marker': mkr} for mkr in marker_list]  # Example, you should provide your own scatter properties
scatter_props_dict = dict(zip(unique_animals, scatter_props))
# {'pin01': {'marker': (5, 0)},
#  'gor01': {'marker': (5, 1)},
#  'vvp01': {'marker': (5, 2)}}
scatter_props_dict

# Pass a function that will return a set of kwargs for a given context
def _return_scatter_props_fn(ctxt: IdentifyingContext):
	""" captures `scatter_props_dict` """
	animal_id = str(ctxt.animal)
	return scatter_props_dict[animal_id]
	


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
_out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

## Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

# Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]

completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths

In [ ]:
# Countable Additivity 
# Any countable collections of points is size 0

